### Práctica de Formatos de datos: XML

La __EMT__ dispone de un portal de datos abiertos que se puede encontrar en la página http://opendata.emtmadrid.es/Home. El objetivo del mismo es ofrecer diferentes tipos de datos sobre la actividad de los autobuses de la EMT. El acceso a los datos se realiza a través de una API de servicios web. 

Para poder utilizar la API de servicios, en primre lugar hay que registrarse, lo cual puede hacerse en la página http://opendata.emtmadrid.es/Formulario. El registro devuelve como resultado en un mensaje electrónico, dos valores:

* Id de cliente: Identificador de cliente

* Pass Key: Password

A continuación, hay que elegir el servicio que se quiere utilizar. Hay 4 servicios definidos: BUS, GEO, MEDIA, INFOPARKING y BICIMAD. Cada servicio tiene asociado un conjunto de métodos que al invocarlos, devuelven un resultado. Por ejemplo, el servicio BUS dispone del servicio __"GetRouteLines"__ que obtiene el itinerario de una línea (o varias líneas separadas por el carácter pipe(|)), con los vértices para construir las rectas del recorrido y las coordenadas UTM de los ejes viales y los códigos de parada. Si se quiere invocar desde Python, se puede hacer usando el siguiente código:

En esta práctica se va a trabajar con varios métodos:

* Método __'GetStreet'__ del servicio GEO. Este método obtiene una lista de emplazamientos EMT coincidentes con una localización. Cada emplazamiento está compuesto por una lista de paradas EMT situadas dentro de un radio predefinido con todos sus atributos, así como las líneas EMT que pasan por cada parada de la lista. En el documento __Servicios_EMT.pdf__ adjunto a esta práctica, se explica con detalle los parámetros de entrada y el resultado que devuelve este método. El método se puede invocar indicando al menos el nombre de la calle y el número:

El método devuelve un documento XML. Entre los elementos del documento, se encuentran las coordenadas X e Y del lugar buscado. En la búsqueda del ejemplo, la calle búscada tiene como coordenadas UTM X e Y:

* Método __'GetStreetRoute'__ del servicio MEDIA. Este método obtiene hasta tres rutas óptimas para ir de un sitio a otro a pie y en autobús. En el documento __Servicios_EMT.pdf__ adjunto a esta práctica, se explica con detalle los parámetros de entrada y el resultado que devuelve este método. El método se puede invocar indicando los siguientes parámetros

  * idClient: Código de cliente autorizado para la operación y suministrado por EMT.

  * passKey: Código de clave asociado al cliente.

  * coordinateXFrom: Campo para identificar la posición "x" del origen de la ruta a consultar. 
  
  * coordinateYFrom: Campo para identificar la posición "y" del origen de la ruta a consultar.

  * coordinateXTo: Campo para identificar la posición "x" del destino de la ruta a consultar. 
  
  * coordinateYTo: Campo para identificar la posición "y" del destino de la ruta a consultar. 

  * criteriaSelection: Campo que indica el criterio de la búsqueda. Los valores son:
  
       * 11 - Mínimo tiempo de trayecto
    
       * 13 - Mínimos trasbordos
    
       * 14 - Mínimo recorrido a pie

El método devuelve un documento XML que contiene información sobre las rutas óptimas (se explica con detalle en el documento __Servicios_EMT.pdf__). En particular muestra la siguiente información:

* El elemento __DescriptionRouteData__ donde aparece la siguiente información:

  * DescriptionDate: Fecha de la ruta.
  * DescriptionInitTime: Descripción de hora de inicio de la ruta.
  * DescriptionEstimateTimeArrival: Descripción de la hora estimada de llegada.
  * Transfers: Trasbordos.
  * LongJourney: Duración del viaje.

* El elemento __ListSectionRoute__ aparece una lista de subelementos __Section__.Cada uno de ellos, describe parcialmente una parte de la ruta. En un caso ideal, aparecerá un subelemento de __Section__ de tipo __WalkingLeg__  que describe el inicio de la ruta andando, a continuación un conjunto de subelementos de __Section__ de tipo __BusLeg__ que describen cada uno de ellos la ruta en una línea de autobus que forma parte de la ruta buscada, y por último otro subelemento de __Section__ de tipo __WalkingLeg__  que describe el final de la ruta andando.

* El elemento __POI__ que muestra información sobre puntos de interés que se encuentran a lo largo de la ruta.

Se pide:

* [1 punto] Crear una función que solicite al usuario dos calles de Madrid, que actúen de origen y destino de una ruta

* [2 puntos]Crear una función que dada una calle de Madrid, devuelva las coordenadas X e Y de la calle.

* [4 puntos]Crear una función que dadas las coordenadas X e Y de un origen y un destino dentro de Madrid, obtenga la ruta más óptima para ir del origen al destino usando autobuses de la EMT. La función deberá imprimir por pantalla, la siguiente información:

 * Fecha de la ruta.
 
 * Hora de inicio de la ruta.
 
 * Hora estimada de llegada
 
 * Número de trasbordos.
 
 * Duración del viaje.
 
 * Descripción textual de la ruta.
 
 Usando el ejemplo anterior debería mostrar:
 
 * Fecha de la ruta: 26/10/2017
 
 * Hora de inicio de la ruta: 03:55
 
 * Hora estimada de llegada: 05:28
 
 * Número de trasbordos: 1
 
 * Duración del viaje: 93
 
 * Descripción textual de la ruta:
 
     * Caminar 42' hasta parada 3125 - EMILIO VARGAS-ARTURO SORIA, linea N4
     
     * 28' en autobús (línea N4) hasta parada 449 - SERRANO-ORTEGA Y GASSET
     
     * Desde parada 449 caminar 5' hasta parada 61 - Castellana-Ministerio Interior, linea N25
 
     * 15' en autobús (línea N25) hasta parada 3691 - SOL-SEVILLA

     * Desde parada 3691 caminar 3'

* [1 punto]Crear una función que solicite al usuarios los nombres de 2 calles de Madrid y muestre la información referente a la ruta más óptima para llegar de una calle a otra usando autobuses de la EMT.

* [2 puntos]Crear una función que solicite al usuarios los nombres de 2 calles de Madrid y muestre un listado de todos los puntos de interés que hay dentro de la ruta más óptima para llegar de una calle a otra usando autobuses de la EMT. A continuación, se le pedirá al usuario que introduzca el nombre de uno de los puntos de interés mostrado, y como resultado se mostrará por pantalla los siguientes datos del punto de interés:

  * Nombre del punto de interés.
  
  * Dirección del punto de interés.
  
  * Teléfono del punto de interés.

In [6]:
from xml.etree import ElementTree
import requests

#Funcion para leer la calle de origen y destino, además llama a la funcion coordenadas,para conseguir las corrdenadas de ambas
def leerCalles():
    #lista donde se guardan coordenadas x y de cada calle
    xyOrigen=[]
    xyDestino=[]
    #leemos la calle origen y comprobamos que no sea vacía
    calleOrigen = input("Introduzca la calle origen:")
    while calleOrigen=="":
        print("No has introducido ninguna calle, vuelve a introducirla")
        calleOrigen = input("Introduzca la calle origen:")
    #hacemos lo mismo para la calle destino
    calleDestino = input("Introduzca la calle destino:")
    while calleDestino=="":
        print("No has introducido ninguna calle, vuelve a introducirla")
        calleDestino = input("Introduzca la calle destino:")
    
    #conseguimos las coordenadas x y de cada calle
    xyOrigen=coordenadas(calleOrigen)
    xyDestino=coordenadas(calleDestino)
    
    #con las corrdenadas, conseguimos la ruta mas optima de la calle origen a destino usando los autobuses de la EMT
    caminoOptimo(xyOrigen,xyDestino)
    
def coordenadas(calle):
    datos = {
        'idClient':'WEB.SERV.segonz09@ucm.es',
        'PassKey':'973F19F0-B74A-407C-82E1-37D96CC9940B',
        'description': calle,
        'streetNumber':'',
        'Radius':'',
        'Stops':'',
        'statistics':'',
        'cultureInfo':''
    }

    url = 'https://servicios.emtmadrid.es:8443/geo/servicegeo.asmx/GetStreet'
    response = requests.post(url, data=datos)
    #lo escribimos en un fichero llamado infoCalle.xml
    ficheroXML = open("infoCalle.xml", 'w')
    ficheroXML.write(response.text)
    ficheroXML.close()
    #lo abrimos y nos sitiamos en la etiqueta Site
    try:
        f= open("infoCalle.xml", "rt")
    except:
        print ("No se ha podido abrir el archivo infoCalle.xml")
        exit()
        
    #lo convertimos en un arbol
    arbol=ElementTree.parse(f)
    nodo = arbol.find('Site')
    #dentro de la etiqueta Site buscamos las etiquetas CoordinateX e CoordinateY y conseguimos su contenido
    coorX = nodo.find('CoordinateX')
    coorY = nodo.find('CoordinateY')
    #guardamos las corrdenadas en una lista y la devolvemos
    xy=[]
    xy.append(coorX.text)
    xy.append(coorY.text)
        
    return xy

def caminoOptimo(xyOrigen,xyDestino):
    datos = {
        'idClient':'WEB.SERV.segonz09@ucm.es',
        'PassKey':'973F19F0-B74A-407C-82E1-37D96CC9940B',
        'statistics':'',
        'cultureInfo':'',
        'coordinateXFrom': xyOrigen[0],
        'coordinateYFrom':xyOrigen[1],
        'originName':'',
        'coordinateXTo': xyDestino[0],
        'coordinateYTo':xyDestino[1],
        'destinationName':'',
        'criteriaSelection':'13',
        'day':'',
        'month':'',
        'year':'',
        'hour':'',
        'minute':'',
        'GenerarAudio':''
    }

    url = 'https://servicios.emtmadrid.es:8443/servicemedia/servicemedia.asmx/GetStreetRoute'
    response = requests.post(url, data=datos)
    #escribimos la guia de como ir de una calle a otra en un fichero xml
    ficheroXML = open("infoRuta.xml", 'w')
    ficheroXML.write(response.text)
    ficheroXML.close()
    
    #lo abrimos y buscamos la etiqueta ListRouteData
    try:
         f= open("infoRuta.xml", "rt")
    except:
        print ("No se ha podido abrir el archivo infoRuta.xml")
        exit()
        
    #lo convertimos en un arbol
    arbol=ElementTree.parse(f)
    listaRutas = arbol.find('ListRouteData') #encontramos la etiqueta ListRouteData
    ruta= listaRutas.find('RouteData') #encontramos la etiqueta RouteData
    descripcion=ruta.find('DescriptionRouteData') #encontramos la etiqueta DescriptionRouteData
    
    #Dentro de DescriptionRouteData cogemos las siguientes etiquetas:
    fecha = descripcion.find('DescriptionDate')
    horaInicio = descripcion.find('DescriptionInitTime')
    horaFin = descripcion.find('DescriptionEstimateTimeArrival')
    trasbordos = descripcion.find('Transfers')
    duracion = descripcion.find('LongJourney')

    #quitamos lo que no valga en el texto
    fecha=fecha.text.replace("Fecha:","")
    horaInicio=horaInicio.text.replace("Hora de Salida:","")
    
    #lo mostramos al usuario
    print("* Fecha de la ruta: "+fecha)
    print("* Hora de inicio de la ruta: "+horaInicio)
    print("* Hora estimada de llegada: "+horaFin.text)
    print("* Número de trasbordos: "+trasbordos.text)
    print("* Duración del viaje: "+duracion.text)
    print("* Descripcion textual de la ruta:\n")
    
    #Dentro de la etiqueta RouteData buscamos la etiqueta ListSectionRoute
    listaSecciones=ruta.find('ListSectionRoute')
    #Recorro todas las secciones
    for seccion in listaSecciones.iter("Section"):
        secciones= list(seccion) #todo el contenido de cada seccion lo meto en una lista
        for paso in secciones:
            #Difrenciamos si hay que ir caminando o en bus
            if paso.tag == "WalkingLeg":
                andar= paso.find('SourceWalkingLeg')
                descripcionRutaAndar=andar.find('RouteDescription')
                #si lo ultimo que se hace es andar,quitamos el " hasta"
                descripcionRutaAndar=descripcionRutaAndar.text.replace(" hasta","")
                print("    *"+descripcionRutaAndar)  
            elif paso.tag == "BusLeg":
                bus= paso.find('SourceBusLeg')
                descripcionRutaBus=bus.find('RouteDescription')
                print("    *"+descripcionRutaBus.text) 
                
#llamada a la funcion, que pide al usuario las calles origen y destino, saca las coordenabas y calcula la ruta mas optima.
leerCalles()

Introduzca la calle origen:Profesor José García Santesmases
Introduzca la calle destino:cascanueces
* Fecha de la ruta: 27/10/2017
* Hora de inicio de la ruta: 11:37
* Hora estimada de llegada: 13:11
* Número de trasbordos: 4
* Duración del viaje: 94
* Descripcion textual de la ruta:

    *Desde  caminar parada 3862 - PARANINFO, linea U
    *7' en autobús (línea U) hasta parada 3863 - PROFESOR GARCIA SANTESMASES-INFORMATICA
    *Desde parada 3863 caminar 3' parada 1697 - PARANINFO-CIEMAT, linea 82
    *13' en autobús (línea 82) hasta parada 1705 - A.FDEZ.CLAUSELLS-ISLA DE OZA
    *Desde parada 1705 caminar 3' parada 1586 - , linea 64
    *Desde parada 1599 caminar 6' parada 1860 - ESTRECHO, linea 43
    *25' en autobús (línea 43) hasta parada 410 - ALFONSO XIII-RAMON Y CAJAL
    *21' en autobús (línea 120) hasta parada 391 - SILVANO-PZA.DEL LICEO
    *Desde parada 391 caminar 16' 
